In [7]:
import pandas as pd
import numpy as np
from gurobipy import *
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [8]:
BUILD_COST = 40
CAPACITY = 1000

In [9]:
k = 30
D = pd.read_csv("./data/2.6/Demand.txt", delimiter="\t", header=None).to_numpy()
kmean = KMeans(k)
kmean.fit(D[:, 1:3])

centers = kmean.cluster_centers_

with open("./data/2.6/Facility.txt", "w") as temp:
    for id, i in enumerate(centers):
        print(id + 1, *np.round(i, 3), CAPACITY, BUILD_COST, file=temp)

with open("./data/2.6/TransCost.txt", "w") as temp:
    for j in centers:
        for i in D:
            print(np.round(np.linalg.norm(j - i[1:3]), 3), end=" ", file=temp)
        print("", file=temp)

In [10]:
f = pd.read_csv("./data/2.6/Facility.txt", delimiter=" ", header=None).dropna(axis=1).iloc[:, 1:].to_numpy()
t = pd.read_csv("./data/2.6/TransCost.txt", delimiter=" ", header=None).dropna(axis=1).to_numpy()
D = pd.read_csv("./data/2.6/Demand.txt", delimiter="\t", header=None).to_numpy()

In [11]:
model = Model("2.6.3")
x = model.addMVar(t.shape, vtype=GRB.CONTINUOUS, name="x")
c = model.addMVar((f.shape[0]), vtype=GRB.BINARY, name="c")
model.update()

model.addConstr(x.sum(axis=1) <= CAPACITY * c)
model.addConstr(x.sum(axis=0) >= D[:, -1])

model.setObjective((c * BUILD_COST).sum() + (x * t).sum(), sense=GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 230 rows, 6030 columns and 12030 nonzeros
Model fingerprint: 0x586255a2
Variable types: 6000 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [9e-03, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 4e+01]
Found heuristic solution: objective 13739.937000
Presolve time: 0.02s
Presolved: 230 rows, 6030 columns, 12030 nonzeros
Variable types: 6000 continuous, 30 integer (30 binary)

Root relaxation: objective 1.512775e+03, 230 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [12]:
if model.status == GRB.Status.OPTIMAL:
    print(0, end=" ")
    for i in range(f.shape[0]):
        print(abs(np.round(c[i].X)), end=" ")
    print()
    for i in range(f.shape[0]):
        if c[i].X:
            for j in range(D.shape[0]):
                if x[i, j].X != 0:
                    print(f"{i + 1} {j + 1} {x[i, j].X}")
    print(model.ObjVal)

0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 
2 114 9.0
2 120 39.0
2 128 13.0
2 140 16.0
2 144 17.0
2 145 26.0
2 146 22.0
2 176 9.0
2 198 7.0
2 199 16.0
3 12 44.0
3 22 20.0
3 74 25.0
3 78 26.0
3 79 42.0
3 93 44.0
3 94 15.0
4 117 13.0
4 122 23.0
4 133 39.0
4 142 22.0
4 150 19.0
4 159 31.0
4 190 16.0
4 192 25.0
6 3 35.0
6 10 15.0
6 14 19.0
6 56 9.0
6 61 29.0
6 62 34.0
6 72 15.0
6 73 17.0
6 83 28.0
6 99 44.0
7 105 44.0
7 115 29.0
7 125 20.0
7 131 28.0
7 141 16.0
7 151 17.0
7 156 15.0
7 171 31.0
7 172 34.0
7 180 26.0
8 134 39.0
8 137 36.0
8 147 26.0
8 155 7.0
8 163 31.0
8 178 13.0
8 179 20.0
8 194 7.0
9 36 36.0
9 107 15.0
9 148 35.0
9 149 42.0
9 157 16.0
9 162 39.0
9 181 36.0
9 185 13.0
9 186 23.0
9 197 15.0
10 101 10.0
10 102 26.0
10 123 39.0
10 138 20.0
10 170 41.0
10 182 42.0
10 183 17.0
10 184 42.0
10 189 7.0
11 11 9.0
11 13 12.0
11 55 7.0
11 59 13.0
11 98 32.0
11 119 31.0
11 164 19.0
11 173 39.0
11 195 39.0
1